# 10-process-data
> Importing, cleaning, testing, and saving data

This series of notebooks reflects operations to scrape, prepare, validate, and save the data.

#### Helpful packages and preliminaries

In [5]:
import sys
from pathlib import Path
import dask.dataframe as dd 
import dask as dk
import pandas as pd 
import pyarrow
import re

sys.path.append(str(Path().resolve().parent))

from src.constants import raw_data_dir, processed_data_dir, raw_data_name

In [3]:
def read_sheet(sheet: int):
    return dk.delayed(pd.read_excel)(raw_data_dir / raw_data_name, sheet_name = sheet)

In [6]:
ddf_factors = read_sheet(1)
ddf_categories = read_sheet(2)

ddf_factors_long = ddf_factors.melt(id_vars = ['Date'], 
                                    var_name = "Variable", 
                                    value_name = 'value')

In [14]:
p = re.compile(r" |&|/")

ddf_all = ddf_factors_long.merge(ddf_categories, on = 'Variable', how = 'left')
ddfs = []
categories = ddf_categories["Category"].unique().compute()
for c in categories:
    factors = ddf_categories.loc[ddf_categories["Category"] == c, "Variable"].tolist()
    ddf = ddf_factors[factors]

    c = p.sub("-", c, 1).replace("& ", "").lower()
    ddfs.append(ddf.to_parquet(processed_data_dir / f"{c}.parquet"))
    
dk.compute(*ddfs)